In [ ]:
%matplotlib widget
import torch
import os
import numpy as np
from sklearn.manifold import TSNE
from umap import UMAP
import matplotlib.pyplot as plt
from torch_geometric.utils import dropout_node


In [ ]:
ckpt_path = '/home/liurzh/ContrastivePE/log_dir/chain/20240519122714/model/model.pt'
triplets_path = '/home/liurzh/ContrastivePE/log_dir/chain/20240519122714/model/triplets.npy'
ckpt = torch.load(ckpt_path, map_location='cpu')
triplets = torch.from_numpy(np.load(triplets_path))
edge_index = triplets[:, :-1].T

ent_x = ckpt['ent_x']
ent_phi = ckpt['ent_phi']
theta = ckpt['theta']

ent = theta * ent_x + ent_phi

f = plt.figure()
adj = torch.zeros(ent_x.shape[0], ent_x.shape[0])
adj[edge_index[0], edge_index[1]] = 1
plt.imshow(adj)
plt.show()

In [ ]:
ent_x_tsne = TSNE(learning_rate='auto', init='random', perplexity=3).fit_transform(ent_x)
ent_phi_tsne = TSNE(learning_rate='auto', init='random', perplexity=3).fit_transform(ent_phi)
ent_tsne = TSNE(learning_rate='auto', init='random', perplexity=3).fit_transform(ent)

In [ ]:
visualizer = UMAP()
ent_x_umap = visualizer.fit_transform(ent_x)
ent_phi_umap = visualizer.fit_transform(ent_phi)
ent_umap = visualizer.fit_transform(ent)

In [ ]:
import matplotlib.cm as cm
visu_data = ent_phi_tsne
colors = ['r', 'g', 'b']
types = [36, 27, 62]
labels = np.random.choice([1,2,3,4], size=(ent_x.shape[0]//2,), replace=True)
chain_label = np.zeros(ent_x.shape[0]//2)
labels = np.concatenate([chain_label, labels])
colors = plt.cm.Set3(np.linspace(0, 1, 5))

fig, ax = plt.subplots()

# fig.set_figheight(10)
# fig.set_figwidth(10)

# for label_i in data.y.unique():
#     cands = data.y == label_i
#     ax.scatter(visu_data[cands, 0], visu_data[cands, 1], alpha=0.3)

for edge in edge_index.T:
    p0, p1 = visu_data[edge[0]], visu_data[edge[1]]
    ax.plot((p0[0], p1[0]), (p0[1], p1[1]), alpha=0.1, color='black')


for label_i, c_i in zip(range(5), colors):  
    ax.scatter(visu_data[labels == label_i, 0], visu_data[labels == label_i, 1], alpha=1, color=c_i, zorder=2.5, label=f'class {label_i}')

# for i, point in enumerate(visu_data):
#     ax.annotate(str(i), (point[0], point[1]))


# plt.title('RotH Entity Embedding Visualization')
# plt.legend()
plt.savefig('chain_vis.pdf')
